## C20P1305 Test Vector Check

### c20p1305 test

In [1]:
plaintext = b'\x4c\x61\x64\x69\x65\x73\x20\x61\x6e\x64\x20\x47\x65\x6e\x74\x6c\x65\x6d\x65\x6e\x20\x6f\x66\x20\x74\x68\x65\x20\x63\x6c\x61\x73\x73\x20\x6f\x66\x20\x27\x39\x39\x3a\x20\x49\x66\x20\x49\x20\x63\x6f\x75\x6c\x64\x20\x6f\x66\x66\x65\x72\x20\x79\x6f\x75\x20\x6f\x6e\x6c\x79\x20\x6f\x6e\x65\x20\x74\x69\x70\x20\x66\x6f\x72\x20\x74\x68\x65\x20\x66\x75\x74\x75\x72\x65\x2c\x20\x73\x75\x6e\x73\x63\x72\x65\x65\x6e\x20\x77\x6f\x75\x6c\x64\x20\x62\x65\x20\x69\x74\x2e'

key = b'\x80\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f\x90\x91\x92\x93\x94\x95\x96\x97\x98\x99\x9a\x9b\x9c\x9d\x9e\x9f'

nonce = b'\x07\x00\x00\x00\x40\x41\x42\x43\x44\x45\x46\x47'

aad = b'\x50\x51\x52\x53\xc0\xc1\xc2\xc3\xc4\xc5\xc6\xc7'

exp_ciphertext = b'\xd3\x1a\x8d\x34\x64\x8e\x60\xdb\x7b\x86\xaf\xbc\x53\xef\x7e\xc2\xa4\xad\xed\x51\x29\x6e\x08\xfe\xa9\xe2\xb5\xa7\x36\xee\x62\xd6\x3d\xbe\xa4\x5e\x8c\xa9\x67\x12\x82\xfa\xfb\x69\xda\x92\x72\x8b\x1a\x71\xde\x0a\x9e\x06\x0b\x29\x05\xd6\xa5\xb6\x7e\xcd\x3b\x36\x92\xdd\xbd\x7f\x2d\x77\x8b\x8c\x98\x03\xae\xe3\x28\x09\x1b\x58\xfa\xb3\x24\xe4\xfa\xd6\x75\x94\x55\x85\x80\x8b\x48\x31\xd7\xbc\x3f\xf4\xde\xf0\x8e\x4b\x7a\x9d\xe5\x76\xd2\x65\x86\xce\xc6\x4b\x61\x16'

exp_mac = b'\x1a\xe1\x0b\x59\x4f\x09\xe2\x6a\x7e\x90\x2e\xcb\xd0\x60\x06\x91'

exp_poly_key = b'\x7b\xac\x2b\x25\x2d\xb4\x47\xaf\x09\xb6\x7a\x55\xa4\xe9\x55\x84\x0a\xe1\xd6\x73\x10\x75\xd9\xeb\x2a\x93\x75\x78\x3e\xd5\x53\xff'


In [2]:
from chacha20poly1305 import ChaCha20Poly1305

cip = ChaCha20Poly1305(key)
ciphertext = cip.encrypt(nonce, plaintext, aad)

# Compare computed ciphertext with expected ciphertext
if ciphertext[:-16] == exp_ciphertext:
    print("Computed ciphertext matches expected ciphertext.")
else:
    print("Computed ciphertext does not match expected ciphertext.")

# Compare computed digest with expected digest
if exp_mac == ciphertext[-16:]:
    print("Computed digest matches expected MAC.")
else:
    print("Computed digest does not match expected MAC.")

Computed ciphertext matches expected ciphertext.
Computed digest matches expected MAC.


## Generating random test vectors for c20p1305 (and chacha20)
The values are saved in 6 files of 512b, 1K, 2K, 4K, 8K, 16K, then used for the check in the GVSoC implementation
Is observed that, except for the last 16 bytes, the test vector is suitable to test chacha20

In [3]:
import os

# Define sizes
sizes = [512, 1024, 2048, 4096, 8192, 16384, 32768]

# Initialize ChaCha20Poly1305 cipher
cip = ChaCha20Poly1305(key)

# Iterate over sizes
for size in sizes:
    # Generate random plaintext
    plaintext = os.urandom(size)

    # Encrypt plaintext
    ciphertext = cip.encrypt(nonce, plaintext, aad)

    # Write plaintext and ciphertext as uint8_t arrays to a text file
    with open(f'testvectorC20P1305{int(size/1024)}K.txt', 'w') as file:
        file.write("const uint8_t plaintext[] = {")
        for byte in plaintext:
            file.write(f"0x{byte:02x}, ")
        file.write("};\n")

        file.write("const uint8_t ciphertext[] = {")
        for byte in ciphertext:
            file.write(f"0x{byte:02x}, ")
        file.write("};\n")

## AES256gcm Test Vector Check

In [4]:
key = (
    b'\xE3\xC0\x8A\x8F\x06\xC6\xE3\xAD\x95\xA7\x05\x57\xB2\x3F\x75\x48'
    b'\x3C\xE3\x30\x21\xA9\xC7\x2B\x70\x25\x66\x62\x04\xC6\x9C\x0B\x72'
)

plaintext = (
    b'\x08\x00\x0F\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1A\x1B\x1C'
    b'\x1D\x1E\x1F\x20\x21\x22\x23\x24\x25\x26\x27\x28\x29\x2A\x2B\x2C'
    b'\x2D\x2E\x2F\x30\x31\x32\x33\x34\x35\x36\x37\x38\x39\x3A\x00\x02'
)

aad = (
    b'\xD6\x09\xB1\xF0\x56\x63\x7A\x0D\x46\xDF\x99\x8D\x88\xE5\x2E\x00'
    b'\xB2\xC2\x84\x65\x12\x15\x35\x24\xC0\x89\x5E\x81'
)

nonce = b'\x12\x15\x35\x24\xC0\x89\x5E\x81\xB2\xC2\x84\x65'

exp_ciphertext = (
    b'\xE2\x00\x6E\xB4\x2F\x52\x77\x02\x2D\x9B\x19\x92\x5B\xC4\x19\xD7'
    b'\xA5\x92\x66\x6C\x92\x5F\xE2\xEF\x71\x8E\xB4\xE3\x08\xEF\xEA\xA7'
    b'\xC5\x27\x3B\x39\x41\x18\x86\x0A\x5B\xE2\xA9\x7F\x56\xAB\x78\x36'
)

exp_tag = (
    b'\x5C\xA5\x97\xCD\xBB\x3E\xDB\x8D\x1A\x11\x51\xEA\x0A\xF7\xB4\x36'
)


In [5]:
from Crypto.Cipher import AES

cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

cipher.update(aad)

ciphertext, tag = cipher.encrypt_and_digest(plaintext)

# Compare computed ciphertext with expected ciphertext
if ciphertext == exp_ciphertext:
    print("Computed ciphertext matches expected ciphertext.")
else:
    print("Computed ciphertext does not match expected ciphertext.")

# Compare computed digest with expected digest
if exp_tag == tag:
    print("Computed digest matches expected one")
else:
    print("Computed digest does not match expected one")

Computed ciphertext matches expected ciphertext.
Computed digest matches expected one


## Generating random test vectors for AES256gcm (and ctr)
The values are saved in 6 files of 512b, 1K, 2K, 4K, 8K, 16K, then used for the check in the GVSoC implementation
Is observed that, except for the tag, the test vector is suitable to test ct

In [7]:
# Define sizes
sizes = [512, 1024, 2048, 4096, 8192, 16384, 32768]

# Iterate over sizes
for size in sizes:

    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

    cipher.update(aad)
    
    # Generate random plaintext
    plaintext = os.urandom(size)

    # Encrypt plaintext
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)

    # Concatenate ciphertext and tag
    ciphertext_and_tag = ciphertext + tag

    # Write plaintext and ciphertext as uint8_t arrays to a text file
    with open(f'testvectorAES256GCM{int(size/1024)}K.txt', 'w') as file:
        file.write("const uint8_t plaintext[] = {")
        for byte in plaintext:
            file.write(f"0x{byte:02x}, ")
        file.write("};\n")

        file.write("const uint8_t ciphertext[] = {")
        for byte in ciphertext_and_tag:
            file.write(f"0x{byte:02x}, ")
        file.write("};\n")